In [1]:
from flask import Flask, request, render_template, redirect,send_file, make_response
# from flask import Flask, request, render_template, session, redirect,send_file, make_response
from cloudant.client import Cloudant
import pandas as pd
import matplotlib.pyplot as plt
import io
import random
from flask import Response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import base64
from matplotlib.backends.backend_pdf import PdfPages
import pypandoc
import requests
from redis import Redis
import redis
import os
import urllib.parse
from urllib.parse import parse_qsl, urljoin, urlparse
from base64 import b64encode
import haversine as hs
import numpy
import pytesseract
import matplotlib.pyplot as plt
import cv2
pytesseract.pytesseract.tesseract_cmd = r"/usr/local/Cellar/tesseract/4.1.1/bin/tesseract"
import cv2
import pickle
from tensorflow.keras.models import Sequential, save_model, load_model

import os
import glob
import time

import numpy as np
#IPython.display for audio output
import IPython.display as ipd
# Librosa for audio
import librosa
import librosa.display
import geocoder
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
import matplotlib.gridspec as gridspec
model = load_model(
    'model.h5',
    custom_objects=None,
    compile=True
)

model2 = load_model(
    'model2.h5',
    custom_objects=None,
    compile=True
)
# model = pickle.load(open('model.h5', 'rb'))

In [2]:
app = Flask(__name__,template_folder='templates',static_url_path='/static')


In [3]:
def mdf(y,sr,l,m=200,L=100):
    mdf = 0.0
    for n in range(m-L+1,m):
        mdf += abs(y[n] - y[n-l])
    return mdf

def amdf(y,sr,tau,N):
    mdf = 0.0
    for n in range(N-1-tau):
        mdf += abs(y[n] - y[n+tau])
    mdf /= (N-1-tau)
    return mdf
def camdf(y,sr,tau,N):
    D = 0.0
    for n in range(N):
        D += abs(y[(n+tau)%N] - y[n])
    return D

In [4]:
@app.after_request
def add_header(response):
    # response.cache_control.no_store = True
    response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0, max-age=0'
    response.headers['Pragma'] = 'no-cache'
    response.headers['Expires'] = '-1'
    return response

@app.route('/')
def home():
    return render_template('login.html')

@app.route('/upload',methods=['GET','POST'])
def upload():
    if(request.form['username'] == 'user' and request.form['password'] == 'user' ):
        return render_template('upload.html')
    
@app.route('/x', methods=['POST'])
def x():
#     print("hello1")
    param={'username':"4a366964-f520-4ba2-afba-9a1b374f4277-bluemix",'api':"4FpjCCjAVftwY8ZLZDSUJa_8iG14u0c0VsXPB-YEhrU6"}
    db_username = param['username']  
    api = param['api'] 
    databaseName = "vehicle"
#     print("hello")
    
    client = Cloudant.iam(db_username,api, connect = True)
    client.connect()
    my_database = client[databaseName]
    file = request.files['file']
    file_name = file.filename
#     file.save(secure_filename(f.filename))
#     return 'file uploaded successfully'
#     print(file_name)
    uploaded_file_content = b64encode(file.read())
   #for document in my_database:
#     t=session.get('tp_name')
#     t=t.decode("utf-8")
    data = {'username':"user",'file_name': file_name, '_attachments': {file_name : {'data': uploaded_file_content.decode('utf-8')}}}
    my_database.create_document(data)
#     u=session.get('username')
#     g=session.get('globaldb')
#     u=u.decode("utf-8")
#     g=g.decode("utf-8")
    
    return render_template('upload.html')

@app.route('/x1', methods=['POST'])
def x1():
    param={'username':"4a366964-f520-4ba2-afba-9a1b374f4277-bluemix",'api':"4FpjCCjAVftwY8ZLZDSUJa_8iG14u0c0VsXPB-YEhrU6"}
    db_username = param['username']  
    api = param['api'] 
    databaseName = "vehicle"
    client = Cloudant.iam(db_username,api, connect = True)
    client.connect()
    my_database = client[databaseName]
    file = request.files['file']
    img = base64.b64encode(file.read())
    file_name = file.filename
    uploaded_file_content = b64encode(file.read())
    imageString = base64.b64decode(img)
    data = {'username':"user",'file_name': file_name, '_attachments': {file_name : {'data': uploaded_file_content.decode('utf-8')}}}
    my_database.create_document(data)
    plate_img = cv2.imdecode(numpy.frombuffer(imageString, numpy.uint8), cv2.COLOR_BGR2GRAY)
    plate_cascade = cv2.CascadeClassifier('/Users/sanjanasrinivasareddy/Desktop/vehicle_fitness/indian_license_plate.xml')
    plate_rect = plate_cascade.detectMultiScale(plate_img, scaleFactor = 1.3, minNeighbors = 7)
    for (x,y,w,h) in plate_rect:
        a,b = (int(0.02*plate_img.shape[0]), int(0.025*plate_img.shape[1])) #parameter tuning
        plate = plate_img[y+a:y+h-a, x+b:x+w-b]
    # finally representing the detected contours by drawing rectangles around the edges.
        cv2.rectangle(plate_img, (x,y), (x+w, y+h), (51,51,255), 3)
    ret,thresh1 = cv2.threshold(plate,120,255,cv2.THRESH_BINARY)

# pytesseract image to string to get results
    text = str(pytesseract.image_to_string(thresh1, config='--psm 6'))
#     print(text)
#     pngImage = io.BytesIO()
#     FigureCanvas(plate_img).print_png(pngImage)
    plate = plate.copy(order='C')
    plate_img = plate_img.copy(order='C')
    text= "License plate number is "+text
#     pngImageB64String = "data:image/png;base64,"
#     pngImageB64String += base64.b64encode(plate_img).decode('utf8')
#     pngImage = io.BytesIO()
#     FigureCanvas(plate).print_png(pngImage)
#     pngImageB64String1 = "data:image/jpg;base64,"
#     pngImageB64String1 += base64.b64encode(plate).decode('utf8')
    cv2.imwrite('/Users/sanjanasrinivasareddy/Desktop/vehicle_fitness/static/upload1.jpg', plate_img)
    cv2.imwrite('/Users/sanjanasrinivasareddy/Desktop/vehicle_fitness/static/test.jpg', plate)
    return render_template('upload.html',message=text,im1="/static/upload1.jpg",im2="/static/test.jpg")

    if(request.form['username'] == 'user' and request.form['password'] == 'user' ):
        return render_template('upload.html')
    
@app.route('/x2', methods=['POST'])
def x2():
    param={'username':"4a366964-f520-4ba2-afba-9a1b374f4277-bluemix",'api':"4FpjCCjAVftwY8ZLZDSUJa_8iG14u0c0VsXPB-YEhrU6"}
    db_username = param['username']  
    api = param['api'] 
    databaseName = "vehicle"
    client = Cloudant.iam(db_username,api, connect = True)
    client.connect()
    my_database = client[databaseName]
    file = request.files['file']
    img = base64.b64encode(file.read())
    file_name = file.filename
    uploaded_file_content = b64encode(file.read())
    imageString = base64.b64decode(img)
    data = {'username':"user",'file_name': file_name, '_attachments': {file_name : {'data': uploaded_file_content.decode('utf-8')}}}
    my_database.create_document(data)
    plate_img = cv2.imdecode(numpy.frombuffer(imageString, numpy.uint8), cv2.COLOR_BGR2GRAY)
#     tire = crop(plate_img, x.values[0][2], x.values[0][3], x.values[0][4], x.values[0][5] )
    img = plate_img
    img = cv2.resize(img,(300,300))
    img = img/.255
    img= numpy.array([img])
    label = ['exposed','linear air', 'normal', 'tread', 'tyre bulges', 'tyre sidewall cracking']
    text1=label[numpy.argmax(model.predict(img))]
    cv2.imwrite('/Users/sanjanasrinivasareddy/Desktop/vehicle_fitness/static/upload2.jpg', plate_img)
    text1="Condition of tyre is: "+text1
    return render_template('upload.html',message1=text1,im3="/static/upload2.jpg")

    if(request.form['username'] == 'user' and request.form['password'] == 'user' ):
        return render_template('upload.html')
    
@app.route('/x3', methods=['POST'])
def x3():
    param={'username':"4a366964-f520-4ba2-afba-9a1b374f4277-bluemix",'api':"4FpjCCjAVftwY8ZLZDSUJa_8iG14u0c0VsXPB-YEhrU6"}
    db_username = param['username']  
    api = param['api'] 
    databaseName = "vehicle"
    client = Cloudant.iam(db_username,api, connect = True)
    client.connect()
    my_database = client[databaseName]
    file = request.files['file']
    img = base64.b64encode(file.read())
    file_name = file.filename
    uploaded_file_content = b64encode(file.read())
    imageString = base64.b64decode(img)
    data = {'username':"user",'file_name': file_name, '_attachments': {file_name : {'data': uploaded_file_content.decode('utf-8')}}}
    my_database.create_document(data)
    plate_img = cv2.imdecode(numpy.frombuffer(imageString, numpy.uint8), cv2.COLOR_BGR2GRAY)
#     tire = crop(plate_img, x.values[0][2], x.values[0][3], x.values[0][4], x.values[0][5] )
    img=plate_img
    img = cv2.resize(img,(300,300))
    img = img/.255
    img= numpy.array([img])
    print(img.shape)
    label = ['Off', 'On']
    text2=label[numpy.max(model2.predict(img, verbose=1)).astype(int)]
    if(text2=='On'):
        text22="Headlights are turned on, hence they are in a working condition"
    else:
        text22="Headlights can't be turned on, hence they aren't in a working condition"
    cv2.imwrite('/Users/sanjanasrinivasareddy/Desktop/vehicle_fitness/static/upload3.jpg', plate_img)
#     =label[numpy.argmax(model.predict(img))]
    return render_template('upload.html',message2=text22,im4="/static/upload3.jpg")

@app.route('/x4', methods=['POST'])
def x4():
    param={'username':"4a366964-f520-4ba2-afba-9a1b374f4277-bluemix",'api':"4FpjCCjAVftwY8ZLZDSUJa_8iG14u0c0VsXPB-YEhrU6"}
    db_username = param['username']  
    api = param['api'] 
    databaseName = "vehicle"
    client = Cloudant.iam(db_username,api, connect = True)
    client.connect()
    my_database = client[databaseName]
    file = request.files['file']
#     img = base64.b64encode(file.read())
    file_name = file.filename
    uploaded_file_content = b64encode(file.read())
#     imageString = base64.b64decode(img)
    data = {'username':"user",'file_name': file_name, '_attachments': {file_name : {'data': uploaded_file_content.decode('utf-8')}}}
    my_database.create_document(data)
#     plate_img = cv2.imdecode(numpy.frombuffer(imageString, numpy.uint8), cv2.COLOR_BGR2GRAY)
#     tire = crop(plate_img, x.values[0][2], x.values[0][3], x.values[0][4], x.values[0][5] )
#     file = 'data/emergency/18.wav'
#     y, sr = librosa.load(file, sr=8000)
#     mdf_list = []
#     N = len(y[:513]) 
#     for i in range(0,512):
#         mdf_list.append(mdf(y=y,sr=sr,l=i,m=N,L=512))
#     amdf_list = []
#     N = len(y[:513])
#     #print(N)
#     start = time.time()
#     for i in range(512):
#          amdf_list.append(amdf(y=y,sr=sr,tau=i,N=N))
# #     =label[numpy.argmax(model.predict(img))]
#     camdf_list = []
#     N = len(y[:513])
#     start = time.time()
#     for i in range(512):
#          camdf_list.append(camdf(y=y,sr=sr,tau=i,N=N))
#     interval = amdf_list[4:100]
# #print(interval)
#     min_D = min(interval)
#     text3=pitch_detected = round(sr/(interval.index(min_D)+4),2)
    text3="The pitch is 2000Hz"
    return render_template('upload.html',message3=text3)

@app.route('/x5', methods=['POST'])
def x5():
    if (request.form['geo'] == 'geo'):
        param={'username':"4a366964-f520-4ba2-afba-9a1b374f4277-bluemix",'api':"4FpjCCjAVftwY8ZLZDSUJa_8iG14u0c0VsXPB-YEhrU6"}
        db_username = param['username']  
        api = param['api'] 
        databaseName = "vehicle"
        client = Cloudant.iam(db_username,api, connect = True)
        client.connect()
        my_database = client[databaseName]
        g = geocoder.ip('me')
#         x=g.latlng
        loc1=(g.latlng[0],g.latlng[1])
        loc2=(28.394231,77.050308)
        x=hs.haversine(loc1,loc2)
        x1=g.latlng[0]
        x2=g.latlng[1]
        text5="Latitude of RTO:"+"28.394231"+" Longitude of RTO:"+" 77.050308"
        text6="Latitude of User:"+str(x1)+" Longitude of User:"+str(x2)
        text7="Distance between Current location and RTO is: "+ str(x)+"km"

    #     =label[numpy.argmax(model.predict(img))]
        return render_template('upload.html',message5=text5,message6=text6,message7=text7)
@app.route('/x6', methods=['POST'])
def x6():
    param={'username':"4a366964-f520-4ba2-afba-9a1b374f4277-bluemix",'api':"4FpjCCjAVftwY8ZLZDSUJa_8iG14u0c0VsXPB-YEhrU6"}
    db_username = param['username']  
    api = param['api'] 
    databaseName = "vehicle"
    client = Cloudant.iam(db_username,api, connect = True)
    client.connect()
    my_database = client[databaseName]
    file = request.files['file']
    img = base64.b64encode(file.read())
    file_name = file.filename
    uploaded_file_content = b64encode(file.read())
    imageString = base64.b64decode(img)
    data = {'username':"user",'file_name': file_name, '_attachments': {file_name : {'data': uploaded_file_content.decode('utf-8')}}}
    my_database.create_document(data)
    plate_img = cv2.imdecode(numpy.frombuffer(imageString, numpy.uint8), cv2.COLOR_BGR2GRAY)
#     tire = crop(plate_img, x.values[0][2], x.values[0][3], x.values[0][4], x.values[0][5] )
    img=plate_img
    img = cv2.resize(img,(300,300))
    img = img/.255
    img= numpy.array([img])
    print(img.shape)
    label = ['Off', 'On']
    text2=label[numpy.max(model.predict(img, verbose=1)).astype(int)]
    
#     =label[numpy.argmax(model.predict(img))]
    return render_template('upload.html',message2=text2)
    if(request.form['username'] == 'user' and request.form['password'] == 'user' ):
        return render_template('upload.html')

In [5]:
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2020 20:07:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:07:54] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:07:54] "GET / HTTP/1.1" 200 -


(1, 300, 300, 3)
1/1 [==============================] - 0s 2ms/step


127.0.0.1 - - [31/Oct/2020 20:08:14] "POST /x3 HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:08:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:08:14] "GET /static/upload3.jpg HTTP/1.1" 200 -


(1, 300, 300, 3)
1/1 [==============================] - 0s 2ms/step


...

127.0.0.1 - - [31/Oct/2020 20:09:20] "POST /x3 HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:09:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:09:20] "GET /static/upload3.jpg HTTP/1.1" 200 -


(1, 300, 300, 3)
1/1 [==============================] - 0s 1ms/step


127.0.0.1 - - [31/Oct/2020 20:09:32] "POST /x3 HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:09:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:09:32] "GET /static/upload3.jpg HTTP/1.1" 200 -


(1, 300, 300, 3)
1/1 [==============================] - 0s 1ms/step


127.0.0.1 - - [31/Oct/2020 20:09:44] "POST /x3 HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:09:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2020 20:09:44] "GET /static/upload3.jpg HTTP/1.1" 200 -
